In [ ]:
Create a test database and table with one million dummy records
============================================================================

In [ ]:

CREATE DATABASE loadtestdb;

use loadtestdb;

CREATE TABLE loadtesttable (
ID INT PRIMARY KEY AUTO_INCREMENT,
Name VARCHAR(255),
Age INT,
Salary DECIMAL(10, 2),
Department VARCHAR(50),
City VARCHAR(100),
Country VARCHAR(100)
);


INSERT INTO loadtesttable (Name, Age, Salary, Department, City, Country)
SELECT
    CONCAT(CHAR(FLOOR(RAND() * 26) + 65), 'Person', n),
    FLOOR(RAND() * 100) + 18,
    ROUND(RAND() * 10000000, 2),
    CASE WHEN RAND() < 0.5 THEN 'IT' ELSE 'Sales' END,
    CASE WHEN RAND() < 0.5 THEN 'New York' ELSE 'Los Angeles' END,
    CASE WHEN RAND() < 0.5 THEN 'USA' ELSE 'Canada' END
FROM (
    SELECT
        a.N + b.N * 10 + c.N * 100 + d.N * 1000 + e.N * 10000 + f.N * 100000 + 1 AS n
    FROM
        (SELECT 0 AS N UNION ALL SELECT 1 UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4 UNION ALL SELECT 5 UNION ALL SELECT 6 UNION ALL SELECT 7 UNION ALL SELECT 8 UNION ALL SELECT 9) a,
        (SELECT 0 AS N UNION ALL SELECT 1 UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4 UNION ALL SELECT 5 UNION ALL SELECT 6 UNION ALL SELECT 7 UNION ALL SELECT 8 UNION ALL SELECT 9) b,
        (SELECT 0 AS N UNION ALL SELECT 1 UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4 UNION ALL SELECT 5 UNION ALL SELECT 6 UNION ALL SELECT 7 UNION ALL SELECT 8 UNION ALL SELECT 9) c,
        (SELECT 0 AS N UNION ALL SELECT 1 UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4 UNION ALL SELECT 5 UNION ALL SELECT 6 UNION ALL SELECT 7 UNION ALL SELECT 8 UNION ALL SELECT 9) d,
        (SELECT 0 AS N UNION ALL SELECT 1 UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4 UNION ALL SELECT 5 UNION ALL SELECT 6 UNION ALL SELECT 7 UNION ALL SELECT 8 UNION ALL SELECT 9) e,
        (SELECT 0 AS N UNION ALL SELECT 1 UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4 UNION ALL SELECT 5 UNION ALL SELECT 6 UNION ALL SELECT 7 UNION ALL SELECT 8 UNION ALL SELECT 9) f
) AS Numbers;

Query OK, 1000000 rows affected (14.62 sec)
Records: 1000000  Duplicates: 0  Warnings: 0


In [ ]:

[root@mysqlprod02 ~]# lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              4
On-line CPU(s) list: 0-3
Thread(s) per core:  1
Core(s) per socket:  1
Socket(s):           4


[root@mysqlprod02 ~]# free -m

              total        used        free      shared  buff/cache   available
Mem:           5709        1599        3066          36        1043        3835
Swap:          3071           0        3071
[root@mysqlprod02 ~]# 


top - 01:51:29 up 8 min,  1 user,  load average: 0.36, 0.29, 0.19
Tasks: 254 total,   1 running, 253 sleeping,   0 stopped,   0 zombie
%Cpu(s):  5.2 us,  1.1 sy,  0.0 ni, 92.9 id,  0.0 wa,  0.8 hi,  0.1 si,  0.0 st
MiB Mem :   5709.6 total,   3060.2 free,   1605.4 used,   1044.0 buff/cache
MiB Swap:   3072.0 total,   3072.0 free,      0.0 used.   3829.0 avail Mem 


mysql> show variables like '%innodb_redo_log%';
+------------------------------+-----------+
| Variable_name                | Value     |
+------------------------------+-----------+
| innodb_redo_log_capacity     | 104857600 |
+------------------------------+-----------+

mysql> show variables like '%innodb_buffer_pool%';
+-------------------------------------+----------------+
| Variable_name                       | Value          |
+-------------------------------------+----------------+
| innodb_buffer_pool_chunk_size       | 134217728      |
| innodb_buffer_pool_instances        | 1              |
| innodb_buffer_pool_size             | 134217728      |
+-------------------------------------+----------------+



To run a concurrency test with 5, 10, and 15 threads, each executing 50 queries 5 times, run the following command
============================================================================================================================


[root@mysqlprod02 ~]# mysqlslap --concurrency=5,10,15 --iterations=5 --number-of-queries=50 
                      --query="SELECT ID, Name, Age, Salary, Department, City, Country FROM loadtesttable WHERE Name like 'A%' AND 
                      Age BETWEEN 30 AND 40;" --create-schema=loadtestdb --verbose
Benchmark
	Average number of seconds to run all queries: 15.041 seconds
	Minimum number of seconds to run all queries: 12.847 seconds
	Maximum number of seconds to run all queries: 19.217 seconds
	Number of clients running queries: 5
	Average number of queries per client: 10

Benchmark
	Average number of seconds to run all queries: 14.087 seconds
	Minimum number of seconds to run all queries: 12.908 seconds
	Maximum number of seconds to run all queries: 15.710 seconds
	Number of clients running queries: 10
	Average number of queries per client: 5

Benchmark
	Average number of seconds to run all queries: 13.244 seconds
	Minimum number of seconds to run all queries: 11.588 seconds
	Maximum number of seconds to run all queries: 15.143 seconds
	Number of clients running queries: 15
	Average number of queries per client: 3


top - 01:55:59 up 13 min,  1 user,  load average: 1.20, 0.43, 0.24
Tasks: 257 total,   2 running, 255 sleeping,   0 stopped,   0 zombie
%Cpu(s): 97.6 us,  0.6 sy,  0.0 ni,  0.9 id,  0.0 wa,  0.8 hi,  0.1 si,  0.0 st
MiB Mem :   5709.6 total,   2854.6 free,   1710.7 used,   1144.3 buff/cache
MiB Swap:   3072.0 total,   3072.0 free,      0.0 used.   3714.5 avail Mem 


top - 01:56:29 up 13 min,  1 user,  load average: 3.28, 1.01, 0.44
Tasks: 257 total,   1 running, 256 sleeping,   0 stopped,   0 zombie
%Cpu(s): 97.8 us,  0.8 sy,  0.0 ni,  0.0 id,  0.0 wa,  1.2 hi,  0.2 si,  0.0 st
MiB Mem :   5709.6 total,   2854.3 free,   1711.0 used,   1144.3 buff/cache
MiB Swap:   3072.0 total,   3072.0 free,      0.0 used.   3714.2 avail Mem 



To run a stress test with 20 threads, each executing the query 10 times, run the following command:
--------------------------------------------------------------------------------------------------------------

[root@mysqlprod02 ~]# mysqlslap --concurrency=20 --iterations=10 --query="SELECT ID, Name, Age, Salary, Department, City, Country FROM
                                (SELECT *, ROW_NUMBER() OVER (PARTITION BY Department ORDER BY Salary DESC) AS R FROM loadtesttable) AS 
                                ranked WHERE R <= 5;" --create-schema=loadtestdb --verbose

Benchmark
	Average number of seconds to run all queries: 61.943 seconds
	Minimum number of seconds to run all queries: 50.947 seconds
	Maximum number of seconds to run all queries: 101.671 seconds
	Number of clients running queries: 20
	Average number of queries per client: 1


top - 02:05:19 up 22 min,  1 user,  load average: 8.93, 4.05, 2.42
Tasks: 255 total,   4 running, 251 sleeping,   0 stopped,   0 zombie
%Cpu(s): 29.5 us, 67.2 sy,  0.0 ni,  0.0 id,  0.0 wa,  3.0 hi,  0.3 si,  0.0 st
MiB Mem :   5709.6 total,   1387.2 free,   1753.0 used,   2569.4 buff/cache
MiB Swap:   3072.0 total,   3072.0 free,      0.0 used.   3670.4 avail Mem 


top - 02:06:02 up 23 min,  1 user,  load average: 16.61, 6.65, 3.37
Tasks: 256 total,   3 running, 253 sleeping,   0 stopped,   0 zombie
%Cpu(s): 84.8 us,  8.1 sy,  0.0 ni,  1.3 id,  0.6 wa,  4.2 hi,  0.9 si,  0.0 st
MiB Mem :   5709.6 total,   1456.0 free,   2095.4 used,   2158.1 buff/cache
MiB Swap:   3072.0 total,   3071.5 free,      0.5 used.   3335.8 avail Mem 


top - 02:06:54 up 23 min,  1 user,  load average: 17.84, 8.58, 4.21
Tasks: 257 total,   3 running, 254 sleeping,   0 stopped,   0 zombie
%Cpu(s): 26.0 us, 34.9 sy,  0.0 ni,  0.0 id, 17.1 wa, 21.0 hi,  1.0 si,  0.0 st
MiB Mem :   5709.6 total,    568.3 free,   1991.9 used,   3149.4 buff/cache
MiB Swap:   3072.0 total,   3070.2 free,      1.8 used.   3438.3 avail Mem 


To run a benchmark test with 10 threads, each executing 100 randomly generated queries with a mixed load type, 
run the following command:
-----------------------------------------------------------------------------------------------------------------------

[root@mysqlprod02 ~]# mysqlslap --concurrency=10,20,30 --iterations=100 --number-of-queries=1000 --auto-generate-sql 
                                --auto-generate-sql-load-type=MIXED --verbose
Benchmark
	Average number of seconds to run all queries: 0.436 seconds
	Minimum number of seconds to run all queries: 0.303 seconds
	Maximum number of seconds to run all queries: 0.917 seconds
	Number of clients running queries: 10
	Average number of queries per client: 100

Benchmark
	Average number of seconds to run all queries: 0.364 seconds
	Minimum number of seconds to run all queries: 0.257 seconds
	Maximum number of seconds to run all queries: 0.949 seconds
	Number of clients running queries: 20
	Average number of queries per client: 50

Benchmark
	Average number of seconds to run all queries: 0.332 seconds
	Minimum number of seconds to run all queries: 0.239 seconds
	Maximum number of seconds to run all queries: 0.592 seconds
	Number of clients running queries: 30
	Average number of queries per client: 33

[root@mysqlprod02 ~]# 


top - 02:21:31 up 38 min,  1 user,  load average: 2.09, 5.74, 7.62
Tasks: 256 total,   2 running, 254 sleeping,   0 stopped,   0 zombie
%Cpu(s): 20.0 us, 25.9 sy,  0.0 ni, 45.1 id,  3.3 wa,  5.0 hi,  0.7 si,  0.0 st
MiB Mem :   5709.6 total,   3112.5 free,   1833.9 used,    763.2 buff/cache
MiB Swap:   3072.0 total,   3065.7 free,      6.2 used.   3601.6 avail Mem 


top - 02:22:40 up 39 min,  1 user,  load average: 5.58, 6.15, 7.64
Tasks: 256 total,   2 running, 254 sleeping,   0 stopped,   0 zombie
%Cpu(s): 20.8 us, 27.9 sy,  0.0 ni, 44.2 id,  3.0 wa,  3.8 hi,  0.3 si,  0.0 st
MiB Mem :   5709.6 total,   3014.1 free,   1848.3 used,    847.1 buff/cache
MiB Swap:   3072.0 total,   3065.7 free,      6.2 used.   3587.2 avail Mem 

